In [62]:
import pandas as pd
from glob import glob 
pd.set_option('max_colwidth', None)
pd.set_option('display.max_rows', None)

In [29]:
files = glob("./spot/*.csv")
spot = [(pd.read_csv(f), f) for f in files]

In [30]:
print(files)

['./spot/U2C_U5A_winter.csv', './spot/U7B_U12B_house.csv', './spot/U7B_U12B_sea.csv', './spot/U3B_U4B_house.csv', './spot/U2C_U5B_sheep.csv', './spot/U7C_U12C_farm.csv', './spot/U3A_U4C_winter.csv', './spot/U3C_U4A_church.csv', './spot/U7C_U12C_house.csv', './spot/U3B_U4B_sheep.csv', './spot/U1B_U6A_jungle.csv', './spot/U3B_U4B_sea.csv', './spot/U1A_U6C_church.csv', './spot/U7C_U12C_jungle.csv', './spot/U7B_U12B_jungle.csv', './spot/U7C_U12C_church.csv', './spot/U7B_U12B_city.csv', './spot/U2C_U5A_beach.csv', './spot/U1A_U6B_house.csv', './spot/U2C_U5B_house.csv', './spot/U1B_U6C_house.csv', './spot/U1B_U6C_sheep.csv', './spot/U7B_U12B_beach.csv', './spot/U1A_U6B_sea.csv', './spot/U1C_U6A_sea.csv', './spot/U3B_U4C_city.csv', './spot/U3C_U4B_winter.csv', './spot/U1C_U6A_house.csv', './spot/U1C_U6C_beach.csv', './spot/U1C_U6B_jungle.csv', './spot/U3A_U4C_beach.csv', './spot/U3C_U4B_farm.csv', './spot/U7B_U12B_sheep.csv', './spot/U7C_U12C_beach.csv', './spot/U7B_U12B_winter.csv', './spot/

In [31]:
spot[0]

(     turn_index               topic  start_time speaker  \
 0             0  checking readiness      537.18     U5A   
 1             1  checking readiness      537.99     U2C   
 2             2  checking readiness      539.02     U2C   
 3             3  describing picture      540.98     U2C   
 4             4  describing picture      541.63     U2C   
 ..          ...                 ...         ...     ...   
 113         106   describing object      727.23     U2C   
 114         107   describing object      729.28     U5A   
 115         108  finding difference      730.01     U2C   
 116         109  finding difference      734.09     U5A   
 117         110  finding difference      735.26     U2C   
 
                                              utterance  end_time     DU ID  \
 0                                                 YEAH    537.37       NaN   
 1                                                RIGHT    538.30       NaN   
 2                                       

In [82]:
import re
import numpy as np
def filtering(text):
    if type(text) == str:
        text = re.sub("%", "", text)
        text = re.sub("-", "", text)
    return text

def filter_spot(df,f):
    df.loc[:,"utterance"] = df["utterance"].apply(filtering)
    return df
spot_filtered = [(filter_spot(d,f),f) for d,f in spot]

for df,f in spot_filtered:
    df.to_csv("./data/spot_bot/"+f.split("/")[-1],index=False)

In [85]:
spot_filtered = [(pd.read_csv(f),f) for f in glob("./data/spot_bot/*.csv")]
for df,f in spot_filtered[30:]:
    print("SPOT :", f)
    for s in list(df["utterance"])[:10]:
        print(s)
    print("SPOT :", f)
    print("\n\n\n\n\n")

SPOT : ./data/spot_bot/U3A_U4C_beach.csv
YEAH
START
<aa> OKAY
I HAVE <aa>
<aa>
OH MY GOD
I HAVE <aa>
THREE
<aa> BLUE
<ahm> OH MY GOD
SPOT : ./data/spot_bot/U3A_U4C_beach.csv






SPOT : ./data/spot_bot/U3C_U4B_farm.csv
SO THERE'S A FARM SCENE
AHA
YEAH SO
DO YOU HAVE <aa>
TWO GREEN TRUCKS?
<aa> FOR ME ONE IS GREEN AND THE OTHER ONE IS PINK
<aa> THE ONE <IN THE> BELOW IS PINK
<aa> SO WE MAY CLICK THERE
YES
AND THEN
SPOT : ./data/spot_bot/U3C_U4B_farm.csv






SPOT : ./data/spot_bot/U7B_U12B_sheep.csv
SO IN THE CENTER OF THE IMAGE ON THE TOP THERE'S A SUN
YES
NEXT
<ON THE> <ON THE LEFT> ON THE RIGHT OF THE IMAGE THERE'S ONE CLOUD
ON THE RIGHT YES
YES AND THIS CLOUD HAS BLUE <aa>
<eh>
IT'S LIKE <THE> THE THICKER LINE IS BLUE AND THE CLOUD IS WHITE
YES <aa> THE SAME HERE AND THE SUN
YES
SPOT : ./data/spot_bot/U7B_U12B_sheep.csv






SPOT : ./data/spot_bot/U7C_U12C_beach.csv
OKAY
YES READY
<aa>
SO WE HAVE <aa>
<aa> TO THE LEFT THERE IS A SUN
YEAH I HAVE THE SUN TOO
YEAH
AND <aa> IT'S SORT

In [86]:
spot_bot = [(pd.read_csv(f),f) for f in glob("./data/spot_bot/*.csv")]
spot_top = [(pd.read_csv(f),f) for f in glob("./data/spot_top/*.csv")]

In [128]:
def make_spot_mid(spot_top, spot_bot):
    res = []
    for ((i,j),(k,l)) in zip(spot_top,spot_bot):

        intersection = set(i["turn_index"]) & set(k["turn_index"])

        mid = i[i["turn_index"].isin(intersection)]
        midk = k[k["turn_index"].isin(intersection)]
        for idx,(u1, u2) in enumerate(zip(mid["utterance"],midk["utterance"])):
            if len(u1) != len(u2):
                if len(u1) > len(u2):
                    mid.loc[idx, "utterance"] = u2


        res += [(mid,"./data/spot_mid/"+j.split("/")[-1])]
    return res

spot_mid = make_spot_mid(spot_top,spot_bot)

In [131]:
for df,f in spot_mid:
    df.to_csv(f,index=False)